In [13]:
import json
from quam_components.components import *

from pathlib import Path
root_folder = Path("./output")
root_folder.mkdir(exist_ok=True)
for key in ["basic", "referenced", "referenced_multifile"]:
    (root_folder / key).mkdir(exist_ok=True)

# Creating QuAM from scratch

The user defines his own QuAM by inheriting from `QuamRoot` (see `components/quam.py`).
Once defined, it can be instantiated as follows:

In [14]:
quam = QuAM()
quam

QuAM(mixers=[], qubits=[], resonators=[], local_oscillators=[], analog_inputs=[], wiring=QuamDictComponent())

We see that five attributes are defined in quam, all empty so far. We can begin populating it with different `QuamComponent`s

In [15]:
num_qubits = 2
for idx in range(num_qubits):
    # Create qubit components
    local_oscillator_qubit = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_qubit)

    mixer_qubit = Mixer(
        id=f"mixer_q{idx}",
        local_oscillator=local_oscillator_qubit,
        port_I=1,
        port_Q=2,
        frequency_drive=5e9,
    )

    transmon = Transmon(
        id=idx,
        xy=XYChannel(
            mixer=mixer_qubit, pi_amp=10e-3, pi_length=40, anharmonicity=200e6
        ),
        z=ZChannel(port=5),
    )
    quam.qubits.append(transmon)

    # Create resonator components
    local_oscillator_resonator = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_resonator)

    resonator_mixer = Mixer(
        id=f"mixer_r{idx}",
        local_oscillator=local_oscillator_resonator,
        port_I=3,
        port_Q=4,
        frequency_drive=5e9,
    )

    readout_resonator = ReadoutResonator(id=idx, mixer=resonator_mixer)
    quam.resonators.append(readout_resonator)

# Create analog inputs
quam.analog_inputs.append(AnalogInput(port=1))
quam.analog_inputs.append(AnalogInput(port=2))

quam

QuAM(mixers=[], qubits=[Transmon(id=0, frequency_01=None, xy=XYChannel(mixer=Mixer(id='mixer_q0', local_oscillator=LocalOscillator(power=10, frequency=6000000000.0), port_I=1, port_Q=2, frequency_drive=5000000000.0, offset_I=0, offset_Q=0, correction_gain=0, correction_phase=0, controller='con1'), pi_amp=0.01, pi_length=40, anharmonicity=200000000.0, pulses=['Xm90', 'X90', 'X180', 'Ym90', 'Y90', 'Y180'], drag_coefficient=0, ac_stark_detuning=0, qubit=...), z=ZChannel(port=5, z_max_frequency_point=None, pulses=['const_flux'], pulse_length=None, pulse_amplitude=None, filter_fir_taps=None, filter_iir_taps=None, offset=0, controller='con1', qubit=...)), Transmon(id=1, frequency_01=None, xy=XYChannel(mixer=Mixer(id='mixer_q1', local_oscillator=LocalOscillator(power=10, frequency=6000000000.0), port_I=1, port_Q=2, frequency_drive=5000000000.0, offset_I=0, offset_Q=0, correction_gain=0, correction_phase=0, controller='con1'), pi_amp=0.01, pi_length=40, anharmonicity=200000000.0, pulses=['Xm90

## Saving and loading quam

We can then save quam to a json file:

In [16]:
quam.save(root_folder / "basic" / "state.json")

Finally, we can also load quam from the same json file

In [18]:
loaded_quam = QuAM.load(root_folder / "basic" / "state.json")

Importantly, the `state.json` file only describes how QuAM is structured, and how the different QuAM components should be initialized. The components themselves are defined in their respective classes.

## Generating QUA config

We can also generate the qua config from quam. This recursively calls `QuamComponent.apply_to_config()` on all quam components.

In [19]:
qua_config = quam.build_config()
json.dump(qua_config, open(root_folder / "basic" / "qua_config.json", "w"), indent=2)

In [7]:
root_folder / "basic" / "qua_config.json"

PosixPath('output/basic/qua_config.json')

# Quam using references

QuAM allows values to reference another part of QuAM by setting its value to a string starting with a colon: `":path.to.referenced.value"`.

As an example why this is useful, we previously hardcoded all output ports. However, grouping everything at a top-level `"wiring"`` makes more sense, so we can have all output ports reference to it.

Below we also add several more references

In [22]:
quam = QuAM()
quam.wiring = {
    "qubits": [{"port_I": 5 * k, "port_Q": 5 * k + 1, "port_Z": 5 * k + 2} for k in range(num_qubits)],
    "resonators": [
        {"port_I": 5 * k + 3, "port_Q": 5 * k + 4} for k in range(num_qubits)
    ],
}

for idx in range(num_qubits):
    # Create qubit components
    local_oscillator_qubit = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_qubit)

    mixer_qubit = Mixer(
        id=f"mixer_q{idx}",
        local_oscillator=f":local_oscillators[{idx}]",
        port_I=f":wiring.qubits[{idx}].port_I",
        port_Q=f":wiring.qubits[{idx}].port_Q",
        frequency_drive=f":qubits[{idx}].frequency_01",
    )
    quam.mixers.append(mixer_qubit)  # TODO fix with reference

    transmon = Transmon(
        id=idx,
        xy=XYChannel(
            mixer=f":mixers[{2*idx}]", pi_amp=10e-3, pi_length=40, anharmonicity=200e6
        ),
        z=ZChannel(port=f":wiring.qubits[{idx}].port_Z"),
        frequency_01=6.1e9,
    )
    quam.qubits.append(transmon)

    # Create resonator components
    local_oscillator_resonator = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_resonator)

    resonator_mixer = Mixer(
        id=f"mixer_r{idx}",
        local_oscillator=f":local_oscillators[{idx}]",
        port_I=f":wiring.resonators[{idx}].port_I",
        port_Q=f":wiring.resonators[{idx}].port_Q",
        frequency_drive=f":resonators[{idx}].frequency_opt",
    )
    quam.mixers.append(resonator_mixer)

    readout_resonator = ReadoutResonator(
        id=idx, mixer=f":mixers[{2*idx+1}]", frequency_opt=5.9e9
    )
    quam.resonators.append(readout_resonator)

# Create analog inputs
quam.analog_inputs.append(AnalogInput(port=1))
quam.analog_inputs.append(AnalogInput(port=2))

We again save quam and the QUA config. The QUA config is identical to previous, but QuAM has changed significantly

In [21]:
quam.save(root_folder / "referenced" / "state.json")

In [26]:
qua_config = quam.build_config()
json.dump(qua_config, open(root_folder / "referenced" / "qua_config.json", "w"), indent=2)

## Separating QuAM into multiple files

Finally, we can also separate parts of QuAM to be placed into a separate file.
This can be especially useful when combined with referencing because we can now have a separate file dedicated to the wiring of the experiment.

Here we point quam to a folder instead of a json file, and we specify that `"wiring"` should go to a separate `wiring.json` file in that folder:

In [27]:
quam.save(root_folder / "referenced_multifile" / "quam", content_mapping={"wiring.json": ["wiring"]})

It can subseuently be loaded as usual

In [12]:
quam = QuAM.load(root_folder / "referenced_multifile" / "quam")

Separating QuAM into multiple files allows the user to revert back to a previous QuAM but keep some parts the same.
For example, the user may want to revert all experimental settings, but the wiring of the setup should of course not change.